# Week 6 – Algorithms
## 6.1 – Searching
### Algorithmic Complexity
You've already seen an abstract explanation of complexity and the tool we use to describe it (big O notation). Now let's see an example in practice. We said before that it is easier to find a single book than to alphabetise a collection of books. Let's explain some common algorithms for *finding* something, called *searching*.

#### Algorithm vs Data Structure
A quick point to note here is that this week's material focuses on *algorithms* – the methods for doing things in code. But when writing an algorithm we inevitably end up using *data*; we have referred to this synergy of two components throughout the unit. Sometimes the choice of data structure will impact our algorithm – sometimes we need to use a specific algorithm to enable us to work with a specific data structure, and sometimes we need to use a specific data structure to enable us to use a specific algorithm! 

With this focus on algorithms, we will assume for the purpose of these algorithms that we are dealing with data that is stored in an *array* – each item is stored sequentially in memory. We have mentioned previously that arrays do not really exist in Python, but we can use a list to simulate them. 

This will make more sense when we get into more detail about memory works next week, where we will also see some alternatives. For now, just remember we are using Python lists, but we are only using non-negative indexing (e.g. `my_list[2]`) not inbuilt methods or other list features. We obviously cannot use `item in my_list` or `my_list.index(…)` for example, this is the algorithm we are trying to write!

### Linear Search
So, suppose you have been given a list, and an item to find within that list, and you need to return a Boolean indicating whether or not the item is in the list. (As mentioned, without using any special features; pretending the list is an array.)

Hopefully the answer you would come up with is to simply go through each item in the list and check if it is equal to the target item. This is called a *linear search*:

In [7]:
def linear_search(in_list, target):
    for i in range(len(in_list)):
        if in_list[i] == target:
            return True
    return False

linear_search([5, 10, 15, 20, 99, 10], 10)

False

Notice the algorithm can end as soon as it finds the item and return `True`. A common mistake in an algorithm like this is trying to write the `return False` line as an `else` statement inside the for loop: obviously this will only ever check the first item of the list and give incorrect results. Hopefully you've seen and written enough Python now to avoid pitfalls like this but keep an eye out – a simple indentation error on the `return False` line makes a big difference!

#### Linear Search Complexity
Now we can analyse the complexity of this algorithm. The performance scales with the size of the input list, which we will call $n$ as tradition dictates. We also need to assume there is an action which *does not* depend on $n$ to use as our measure of a “unit” piece of complexity, so we can count how often it occurs. In this case, the action will be *accessing* the list item and comparing it to the target. One reason we use arrays/lists is because they actually do have constant time access that does not depend on their size – more about this next week.

So for a given list of size $n$ how many times do we access an item from the list and compare it to the target? Well, this depends on if we get lucky! 

If the first item in the list is our target, then we will only look at one item, no matter how big the list is. So we get *best case complexity* of $O(1)$. Best case complexity is rarely useful: as the list gets bigger this becomes less and less likely. We do not want to rely on luck in practice! 

In the *worst case* the item either isn't in the list, or it is the very last element, either way we check every item in the list. So for a list of size $n$ we make $n$ checks: worst case complexity $O(n)$.

Worst case might seem pessimistic, why not try to estimate the *average case*? This seems to depend on how often we search for an item that isn't in the list. Suppose the probability of an item being *in* the list is equal to $\alpha$. So if $\alpha = 0.5$, we expect half of the searches to be in the list, half to be missing.

If the item is in the list, we can expect it to be equally likely to appear in any position, therefore the average case will be equal to a position half way along the list: $\frac{n}{2}$ comparisons.

If the item isn't in the list, we already know it takes $n$ comparisons.

So the weighted average of these values is equal to $\alpha \frac{n}{2} + (1-\alpha) n$ comparisons. This isn't doing anything complicated, just averaging the two according to the proportions. If $\alpha = 1$ that means all the items are in the list and we get a total of $\frac{n}{2}$ again, as we'd expect.

Now let's put that in big O notation and do some simple algebra: 

\begin{align}
O\left(\alpha \frac{n}{2} + (1-\alpha) n\right) &= O\left(\alpha \frac{n}{2} + n - \alpha n\right) \\
&= O\left(\frac{\alpha n + 2n - 2\alpha n}{2}\right) \\
&= O\left(\frac{(2 - \alpha)}{2}n\right) \\
\end{align}

But remember that in big O notation we ignore constant factors that do not depend on $n$, which $\frac{(2 - \alpha)}{2}$ certainly is. So we get:

\begin{equation}
= O(n)
\end{equation}

So in our worst case *and* our average case we get the same big O result: $O(n)$. This is also called “linear” complexity, hence the name linear search.

#### Average or Worst Case?
There is no simple answer about which is better to use in practice, average case or worst case complexity analysis. Usually it is *easier* to do worst case, because we do not have to make complicated assumptions about the distribution of inputs. This is especially important when the average case ends up being the same as the worst case for extra work, which is the case for most of the examples we will be looking at. This is partly why big O notation is a useful tool. Obviously on average we expect an algorithm to perform better than in the worst case, but it may *scale* the same or essentially the same.

For more complicated situations where you know a more accurate distribution, average case analysis might be useful. In general we will use worst case analysis most of all. But if a particular algorithm happens to have a better average case complexity but the same worst case, then this is worth mentioning and considering when comparing the algorithms.

#### Membership vs Index
The code above searches for an item then simply returns *if* the item is in the list as a Boolean. But we might want to know *where* the item exists in the list instead. It is a simple change to the code to make this return the index if the item is found. This requires two considerations:
* If the item is not in the list, we should still return something. `False` might get confused with `0` which is a valid position, so we should avoid this. We could use `-1` instead – in many languages this is not a valid index position, but in Python it is! So the person using the function must be careful. `None` is an option, so is simply raising an error.
* If the list can contain duplicates then we must decide which index to return: usually the first, but we could modify the algorithm to return a different index, or all possible indices.

***Exercise:*** modify the code above to return the index rather than a membership Boolean.

### Binary Search
It's not much good analysing a single algorithm with nothing to compare it to. Enter the next search algorithm: *binary search*.

Binary search is comparable to how you'd search a phone book for a name (if you are old enough to remember what a [physical phone book](https://en.wikipedia.org/wiki/Telephone_directory) is). You start about half way, if your target name is *earlier* in the alphabet than the ones on the page then you need to look in the pages earlier (to the left) in the book. If the target is *later* alphabetically than the ones on the page, then you need to look later in the book. You can keep repeating this process, going half way each time, until you find the name.

The trick is that binary search requires a *sorted list*, i.e. a list where the items are in order. Hopefully it's obvious why: if you are looking for a name starting with "E", then open the phone book half way and see a name starting with "M", you can only turn to a page to the left if you know the names are in order.

To actually put this into practice, the best method is to keep track of the start and end indices of your current search. You can think of these as values that are pointing to positions in the list. Let's call them `start_index` and `end_index`, with the idea being that at any given time when we are searching `my_list`, we are actually searching the values between `my_list[start_index]` and `my_list[end_index]` inclusive. These start at `0` and `len(my_list)-1` respectively – i.e. the whole list. Then we find the middle position between these, and see if it's our target. If it isn't, we search the left hand side or the right hand side (excluding the middle element) by updating the necessary index variable.

Here is an implementation in Python:

In [2]:
def binary_search(my_list, target, left_index=None, right_index=None):
    if left_index is None:
        left_index = 0
    if right_index is None:
        right_index = len(my_list)-1
        
    if left_index > right_index:
        return False
    
    middle = (left_index + right_index) // 2
    
    if target == my_list[middle]:
        return True
    
    if target < my_list[middle]:
        return binary_search(my_list, target, left_index=left_index, right_index=middle-1)
    
    assert(target > my_list[middle])
    return binary_search(my_list, target, left_index=middle+1, right_index=right_index)
    

print(binary_search([1, 4, 9, 10, 15], 4))
print(binary_search([1, 4, 9, 10, 15], 20))

True
False


The code above is written recursively, because the algorithm itself has a nice recursive nature to it. Actually it is easier to write another recursive version of this code that does not use the two index variables:

In [3]:
def binary_search_2(my_list, target):
    if len(my_list) == 0:
        return False
    
    middle = (len(my_list)-1) // 2
    
    if target == my_list[middle]:
        return True
    
    if target < my_list[middle]:
        return binary_search(my_list[:middle], target)
    else:
        return binary_search(my_list[middle+1:], target)
    
print(binary_search_2([1, 4, 9, 10, 15], 4))
print(binary_search_2([1, 4, 9, 10, 15], 20))

True
False


The problem is that this version creates a new list object every time we use the *slice* syntax `my_list[a:b]`:

In [4]:
my_list = [1, 2, 3, 4, 5]
my_slice = my_list[2:4]
my_slice[0] = 10

print(my_list)

[1, 2, 3, 4, 5]


Above, the variable `my_slice` is not a *view* into the original list data, it is an entirely new list.

Don't worry if this is getting a bit complicated. The point is that this second version of the binary search uses more memory. We aren't too worried about memory right now, we're looking at time complexity, but if we can avoid using extra memory we probably should.

Another reason to prefer the version that uses the two `index` variables is that it might be more obvious how this is translated into a `while` loop rather than using recursion. ***Exercise:*** convert `binary_search(…)` to use a `while` loop.

#### Binary Search Complexity
So how is the time complexity of binary search? Have a think, can you work out how the behaviour changes with the size of the input list $n$?

We again assume that the comparison cost is fixed, so we are essentially asking how many time the recursion (or loop) repeats. Again, we'll look at worst case complexity, which is simpler than average case.

The easy way to think about it is that the list *halves* in size on each run. This isn't *strictly* true, we can do better than half by ignoring the middle element when we find the next list to search. But halving a list, say from length 1000000 to 500000, is basically the same as going to 499999. If you do the maths of the exact analysis, you'll get the same result.

Suppose the list had length 1024, if we halve over and over we'll get 1024, 512, 256, 128, 64, 32, 16, 8, 4, 2, 1. If it isn't in the last list, it must not be in the list at all. So the worst case number of runs is 10.

The more mathematical of you will recognise this is a logarithmic sequence. $\log_2(1024) = 10$ – because $2^10 = 1024$. If the list isn't a perfect power of two, this won't be an integer number of iterations. But again, complexity is not about being exact. Big O notation is asymptotic, an upper bound. In fact, the difference between $log_2(n)$ and $log_3(n)$ is actually a *constant factor* (think back to the [logarithmic identities](https://en.wikipedia.org/wiki/List_of_logarithmic_identities#Changing_the_base) to see why). So when we see logarithmic complexity we do not care about base, we just write $O(\log(n))$.

Logarithmic complexity is *much* better than linear complexity. To illustrate, imagine you had a list of a trillion items. Linear search would obviously have a worst case in the order of one trillion, whereas binary search complexity would be in the order of *forty*. Not forty thousand, just forty! Obviously, this requires the list is sorted, the subject of our next section.

If you are designing an algorithm, linear complexity is still very good. It is not unusual to see algorithms with quadratic complexity or worse, as you will also see in the next section.

## What's Next?
Once you are done with the material (and exercises) on this page, go back to Engage and move onto the next section.